<a href="https://colab.research.google.com/github/fora22/Data_Science/blob/main/Support_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서포트 벡터 머신(Support Vector Machine)
서포트 벡터 머신은 선형이나 비선형 분류, 회귀, 이상치 탐색에도 사용할 수 있는 다목적 머신러닝 모델이다. 특히 복잡한 분류 문제에 잘 들어맞으며 작거나 중간 크기의 데이터셋에 적합하다.

서포트 벡터 머신은 결정 경계(Decision Boundary), 즉 분류를 위한 기준 선을 정의하는 모델이다. 다른 머신러닝 분류 모델 또한 결정 경계를 통해 데이터를 분류하지만 서포트 벡터 머신의 특징으로는 '마진'이라는 개념이 있다.

# 마진(Margin)
![image](https://user-images.githubusercontent.com/48875566/127726733-c4d7af70-6ca1-4a02-b8be-4939f594af3b.png)
* [그림 출처](http://hleecaster.com/ml-svm-concept/)

그림에서 실선이 하나 그어져 있는데 이것이 '결정 경계'이다. 그리고 그 실선으로부터 검은 테두리가 있는 빨간점 1개, 파란점 2개까지 영역을 두고 점섬은 그려놓았는데, 결정 경계에서 점선까지의 거리가 바로 마진이다. 그리고 서포트 벡터 머신은 마진을 최대화시켜 최적의 결정 경계를 도출한다.

그림에서 x축과 y축 2개 속성을 가진 데이터로 결정 경계를 그릴 때 3개의 서포트 벡터(선)가 필요했다. 즉, $n$개의 속성을 가진 데이터에는 최소 $n+1$개의 서포트 벡터가 존재한다.


# 소프트 마진 분류
모든 샘플이 도로(선) 바깥쪽에 올바르게 분류되어 있다면 이를 **하드 마진 분류(hard margin classfication)**라고 한다. 이는 두 가지 문제점이 있는데

1. 데이터가 선형적으로 구분될 수 있어야 제대로 작동하며
2. 이상치에 민감하다.

![image](https://user-images.githubusercontent.com/48875566/127726919-7e72cc34-88a0-4bfe-8adc-05ba057d42c6.png)
* [사진 출처](https://analysis-flood.tistory.com/94)

왼쪽 그림에서 처럼 이상치가 존재하면 하드 마진 분류는 불가능하다. 따라서 이런 문제를 피할려면 유연한 모델이 필요하다. 

도로(선)의 폭을 가능한 한 넓게 유지하는 것과 마진 오류(margin violation : 샘플이 도로 중간이나 심지어 반대쪽에 있는 경우) 사이에 적절한 균형을 잡아야 한다. 이를 소프트 마진 분류(soft margin classfication)이라고 한다.



![image](https://user-images.githubusercontent.com/48875566/127727046-bac7d264-43af-49ca-88e5-14fccb6b4d40.png)
* [그림 출처](https://analysis-flood.tistory.com/94)

sklearn의 SVM 모델에서는 $C$ 하이퍼파라미터를 사용해 이 균형을 조절할 수 있다. $C$ 값을 줄이면 도로의 폭이 넓어지지만 마진 오류도 커진다. 반대로 $C$ 값을 크게 하면 마진 오류가 적지만 도로의 폭이 좁아진다.

두 그림을 비교했을 떄 $C=1$일 경우가(마진이 넓을 때, 도로 폭이 넓을 때) 더 잘 일반화될 것이다. 사실 대부분의 마진 오류는 **결정 경계를 기준**으로 올바른 클래스로 분류되기 때문에 이 훈련 세트에서 예측 에러는 마진 오류보다 작다.



In [1]:
# 그래프 한글 폰트 설정
!apt-get update -qq
!apt-get install fonts-nanum* -qq
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname = path, size = 10).get_name()
print(font_name)
plt.rc('font', family=font_name)  
fm._rebuild()

Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
NanumBarunGothic


In [2]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [3]:
iris = datasets.load_iris()

In [4]:
X = iris["data"][:, (2, 3)] # 꽃잎 길이, 꽃잎 너비
y = (iris['target'] == 2).astype(np.float64)  # Iris-Virginica

In [5]:
svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss='hinge'))
                    ])

In [6]:
svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [7]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

In [8]:
# 만약 데이터 셋이 아주 커서 메모리에 적재할 수 없거나(외부 메모리 훈련), 온라인 학습으로 분류 문제를 다룰 때는 속도를 조금 포기하고 SGDClassfier()를 사용할 수 있다.
# SGDClassfier(loss = 'hinge', alpha = 1/(m*C))

# 비선형 SVM 분류
선형적으로 분류할 수 없을 때는 여러 가지 방법이 있다.

일단 첫 번째로 다항 특성을 다룰때처럼 사용하는 방법이다.
* Polynomial Regression.ipynb 참고

In [9]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [10]:
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

In [12]:
polynomial_svm_clf = Pipeline([
                               ("poly_features", PolynomialFeatures(degree=3)),
                               ("scaler", StandardScaler()),
                               ("svm_clf", LinearSVC(C=10, loss="hinge"))
                               ])

In [13]:
polynomial_svm_clf.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)